In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

In [6]:
#  Imports
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Embedding, MultiHeadAttention
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from datasets import load_dataset


In [7]:
# Paramètres de base du modèle

embed_dim = 512 # (Dimension de l'embedding)
num_heads = 8 # (nombre de mot important dans un phrase)
ff_dim = 2048 # (dimension du PMC)
maxlen = 200 # (mot total dans une phrase)
vocab_size = 10000 # (taile du vocabulaire)
batch_size = 32 # Batch size

In [8]:
# Chargement du dataset "emotion"
dataset = load_dataset("emotion")

train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

x_train = tokenizer.texts_to_sequences(train_texts)
x_test = tokenizer.texts_to_sequences(test_texts)

x_train = pad_sequences(x_train, maxlen=maxlen, padding='post')
x_test = pad_sequences(x_test, maxlen=maxlen, padding='post')

y_train = np.array(train_labels)
y_test = np.array(test_labels)


In [10]:
# TransformerBlock (bloc d'encodage)
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [11]:
# Modèle Transformer complet
class Transformer(tf.keras.Model):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim, rate=0.1):
        super(Transformer, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.trans_block = TransformerBlock(embed_dim, num_heads, ff_dim, rate)
        self.final_layer = Dense(6, activation='softmax')

    def call(self, inputs, training=False):
        x = self.embedding(inputs)
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = x + positions
        x = self.trans_block(x, training=training)
        x = x[:, -1, :]
        return self.final_layer(x)


In [12]:
 # Compilation et entraînement
model = Transformer(vocab_size, embed_dim, num_heads, ff_dim)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=3, validation_data=(x_test, y_test))


Epoch 1/3
500/500 ━━━━━━━━━━━━━━━━━━━━ 7228s 14s/step - accuracy: 0.2929 - loss: 2.2562 - val_accuracy: 0.3475 - val_loss: 1.5654
Epoch 2/3
500/500 ━━━━━━━━━━━━━━━━━━━━ 7156s 14s/step - accuracy: 0.5808 - loss: 1.1340 - val_accuracy: 0.8035 - val_loss: 0.5512
Epoch 3/3
500/500 ━━━━━━━━━━━━━━━━━━━━ 7639s 15s/step - accuracy: 0.8751 - loss: 0.3827 - val_accuracy: 0.8980 - val_loss: 0.2686


In [20]:
model.save("mon_modele_transformer.h5")


In [ ]:
# from tensorflow.keras.models import load_model

# # Charger le modèle .h5
# model = load_model("D:/Projet_Deep_Learning/emotion_transformer_project/Models/mon_modele_transformer.h5")

In [14]:
# Fonction pour prédire une émotion
def predict_emotion(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=maxlen, padding='post')
    prediction = model.predict(padded)
    predicted_label = np.argmax(prediction)
    label_names = dataset['train'].features['label'].names
    print("Texte :", text)
    print("Émotion prédite :", label_names[predicted_label])


In [15]:
# Test de prédiction
predict_emotion("I feel so happy and excited today!")
predict_emotion("I'm scared and I don't know what to do.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step
Texte : I feel so happy and excited today!
Émotion prédite : joy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
Texte : I'm scared and I don't know what to do.
Émotion prédite : anger


In [16]:
predict_emotion("What a beautiful day, I can't stop smiling!")
predict_emotion("I just got promoted, I'm so proud of myself.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
Texte : What a beautiful day, I can't stop smiling!
Émotion prédite : sadness
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
Texte : I just got promoted, I'm so proud of myself.
Émotion prédite : joy


In [17]:
predict_emotion("I feel like crying, everything is falling apart.")
predict_emotion("Nothing brings me joy anymore, I feel empty.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Texte : I feel like crying, everything is falling apart.
Émotion prédite : fear
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Texte : Nothing brings me joy anymore, I feel empty.
Émotion prédite : sadness


In [18]:
predict_emotion("i hate those fuckers")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Texte : i hate those fuckers
Émotion prédite : anger


In [ ]:
import collections
print(collections.Counter(y_train))

Counter({np.int64(1): 5362, np.int64(0): 4666, np.int64(3): 2159, np.int64(4): 1937, np.int64(2): 1304, np.int64(5): 572})
